In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from functions import get_ohclv_df, get_fc_df

# 日频多因子数据路径
PATH_DAY_FREQ_FACTOR_DATA = 'data\\factor_mainforce\\'
# 日频股票OHCLV价格数据路径
PATH_DAY_FREQ_STOCK_DATA = 'data\\day_frequence_data\\'

N_COM = 3
WINDOW = 150
# 设置调仓频率f
FREQ = 30
TRAIN_PORTION = 0.8
INITIAL_MONEY = 1000000.0
# HOLDING_NUM = 10

fc_df = get_fc_df(PATH_DAY_FREQ_FACTOR_DATA)
fc_df = fc_df.reset_index(drop=True)
fc_dfg = fc_df.groupby(['time', 'code']).mean()
fc_dfg.fillna(0.0, inplace=True)
fc_dfg

fc_df is <class 'pandas.core.frame.DataFrame'>


QUA          STD      SKEW      KURT       MTS  \
time       code                                                               
2019-01-30 000001.XSHE  0.140087  1402.432693 -0.375838 -0.684704  0.243028   
           000002.XSHE  0.143918  1451.208475 -0.367501 -0.684472  0.247227   
           000004.XSHE  0.014050    85.942365  0.600557 -0.844007  0.076434   
           000005.XSHE  0.064183   434.749670 -0.078920 -1.017302  0.088695   
           000006.XSHE  0.053757   504.116232  0.086710 -0.971077  0.132394   
...                          ...          ...       ...       ...       ...   
2019-01-29 603993.XSHG  0.143836   833.575336 -0.406125 -0.591979  0.135252   
           603996.XSHG  0.048106   267.349467  0.067295 -1.055960  0.096276   
           603997.XSHG  0.064664   338.470398  0.030480 -0.825601  0.099840   
           603998.XSHG  0.020937   146.203036  0.266222 -0.980688  0.114165   
           603999.XSHG  0.080670   415.661325 -0.104563 -1.000557  0.088721   

                             MTE        SR  
time       code                             
2019-01-30 000001.XSHE  0.095476  0.112201  
           000002.XSHE  0.000286  0.084823  
           000004.XSHE -0.000292 -0.034447  
           000005.XSHE  0.023842  0.229402  
           000006.XSHE  0.042559 -0.002832  
...                          ...       ...  
2019-01-29 603993.XSHG  0.061758  0.102369  
           603996.XSHG  0.049350  0.020826  
           603997.XSHG -0.035953 -0.032998  
           603998.XSHG -0.020632 -0.086087  
           603999.XSHG  0.028537  0.089964  

[3085141 rows x 7 columns]

In [2]:
price_df = get_ohclv_df(PATH_DAY_FREQ_STOCK_DATA)
price_df.fillna(0.0, inplace=True)
price_df

price_df is <class 'pandas.core.frame.DataFrame'>


code,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,688787.XSHG,688788.XSHG,688789.XSHG,688793.XSHG,688798.XSHG,688799.XSHG,688800.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
time,,,,,,,,,,,,,,,,,,,,,
2019-01-02,1079.71,3169.04,119.58,27.30,229.05,97.89,103.68,35.78,43.60,34.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-01-03,1090.29,3191.59,121.00,27.40,229.05,94.39,102.61,35.28,43.60,34.32,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-01-04,1145.51,3305.62,123.60,28.11,233.94,96.44,104.48,36.36,44.13,34.90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-01-07,1144.33,3321.53,128.22,29.03,237.50,97.16,108.76,37.02,45.20,35.82,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-01-08,1134.93,3314.90,125.17,28.93,236.61,94.63,110.63,37.61,44.80,35.56,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-22,1522.85,2809.30,67.39,18.09,219.60,95.48,61.83,126.01,44.80,42.52,...,51.77,57.57,166.77,36.44,95.40,31.63,139.55,37.07,38.45,38.91
2022-09-23,1522.84,2812.51,64.48,18.20,220.62,95.48,61.29,122.69,45.86,41.96,...,52.07,55.89,158.13,34.91,91.03,30.78,132.95,35.66,38.14,38.12
2022-09-26,1486.92,2801.27,62.10,17.28,214.54,92.22,59.68,120.68,44.00,40.51,...,50.22,53.37,155.80,36.56,95.42,30.46,136.40,36.34,37.47,38.60


In [3]:
temp1 = fc_dfg.reset_index()
temp1 = temp1.groupby(['time']).mean().reset_index()
temp1 = temp1['time'].astype('str')
temp2 = price_df.reset_index()
temp2 = temp2.groupby(['time']).mean().reset_index()
temp2 = temp2['time'].astype('str')
temp = pd.merge(temp1, temp2, on=['time'])
temp
date_list = np.array(temp)
date_list = date_list.reshape(766).tolist()
print(f'len(date_list)={len(date_list)}')
# date_list

len(date_list)=766


In [4]:
price_df.keys()
temp3 = fc_dfg.reset_index().groupby(['code']).mean().reset_index()['code']
list1 = temp3.to_list()
list2 = price_df.keys().tolist()
stock_list = [stock for stock in list1 if stock in list2]
print(f'len(stock_list)={len(stock_list)}')
# print(type(price_df.keys()))

len(stock_list)=4726


In [5]:
fc_dfgr = fc_dfg.reset_index()
price_dfr = price_df.reset_index()
fc_dfgr['time'] = pd.to_datetime(fc_dfgr['time'])
# fc_dfgr.fillna(0.0000001, inplace=True)

price_dfr['time'] = pd.to_datetime(price_dfr['time'])
# price_dfr.fillna(0.0000001, inplace=True)
price_dfr

code,time,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,...,688787.XSHG,688788.XSHG,688789.XSHG,688793.XSHG,688798.XSHG,688799.XSHG,688800.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
0,2019-01-02,1079.71,3169.04,119.58,27.30,229.05,97.89,103.68,35.78,43.60,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-01-03,1090.29,3191.59,121.00,27.40,229.05,94.39,102.61,35.28,43.60,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-01-04,1145.51,3305.62,123.60,28.11,233.94,96.44,104.48,36.36,44.13,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-01-07,1144.33,3321.53,128.22,29.03,237.50,97.16,108.76,37.02,45.20,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-01-08,1134.93,3314.90,125.17,28.93,236.61,94.63,110.63,37.61,44.80,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,2022-09-22,1522.85,2809.30,67.39,18.09,219.60,95.48,61.83,126.01,44.80,...,51.77,57.57,166.77,36.44,95.40,31.63,139.55,37.07,38.45,38.91
906,2022-09-23,1522.84,2812.51,64.48,18.20,220.62,95.48,61.29,122.69,45.86,...,52.07,55.89,158.13,34.91,91.03,30.78,132.95,35.66,38.14,38.12
907,2022-09-26,1486.92,2801.27,62.10,17.28,214.54,92.22,59.68,120.68,44.00,...,50.22,53.37,155.80,36.56,95.42,30.46,136.40,36.34,37.47,38.60
908,2022-09-27,1505.50,2826.97,63.81,17.48,217.58,94.87,61.29,122.19,44.93,...,51.26,55.56,164.68,40.27,95.70,31.32,135.96,37.03,38.37,39.68


In [6]:
from datetime import datetime
import pickle
import torch
import torch.nn as nn
from torch.autograd import Variable

def cal_temp_x_y_all(fc_dfgr, price_dfr, date):
    temp_x_all = fc_dfgr[(fc_dfgr['time'] >= datetime.strptime(date_list[date - WINDOW], '%Y-%m-%d'))
                & (fc_dfgr['time'] < datetime.strptime(date_list[date], '%Y-%m-%d'))]
            # & (fc_dfgr['code'] == '000001.XSHE')]
    temp_y_all = price_dfr[(price_dfr['time'] >= datetime.strptime(date_list[date], '%Y-%m-%d'))
                & (price_dfr['time'] < datetime.strptime(date_list[date + FREQ], '%Y-%m-%d'))]
    return temp_x_all, temp_y_all

PCA_MODEL = PCA(n_components=N_COM)
with open(f'lasso_model/lasso_w{WINDOW}_f{FREQ}_tp{TRAIN_PORTION}_ncoms{N_COM}.pickle', 'rb') as file:
    Lasso_MODEL = pickle.load(file)
device = torch.device(0)
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = self.init_hidden_lstm()

        self.fc = self.init_hidden()

    def init_hidden(self):
        return (nn.Linear(self.hidden_size,self.num_classes).to(device)).to(device)

    def init_hidden_lstm(self):
        return (nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size,
                            num_layers=self.num_layers, batch_first=True)).to(device)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)

        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)

        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))

        h_out = h_out.view(-1, self.hidden_size).to(device)

        out = self.fc(h_out).to(device)

        return out
LSTM_MODEL = torch.load('lstm_model/lstm_temp_4_mse_0.000000_epoch70000.pth')
sc = MinMaxScaler()

In [7]:
# for stock in stock_list:
def cal_prediction(ml_model, pca_model, temp_x_all, temp_y_all, stock='000001.XSHE', model_type='lasso'):
    temp_x = temp_x_all[temp_x_all['code'] == stock]  # stock_list[stock] = '000001.XSHE'
    temp_x = temp_x[['QUA', 'STD', 'SKEW', 'KURT', 'MTE', 'SR']]
    temp_x = np.array(temp_x)
    # print(temp_x.shape)
    if temp_x.shape[0] != WINDOW:
        return 0.0
    # print(f'temp_x{temp_x}')
    temp_x = sc.fit_transform(temp_x)
    pca_model = pca_model.fit(temp_x)
    temp_x = pca_model.transform(temp_x)
    temp_y = np.array(temp_y_all[stock])
    temp_y = sc.fit_transform(temp_y.reshape([temp_y.shape[0], 1]))

    if model_type == 'lasso':
        temp_x = temp_x.reshape([1, temp_x.shape[0] * temp_x.shape[1]])
        # print(f'temp_x.shape{temp_x.shape}')
        # temp_x

        temp_y = temp_y.reshape([1, temp_y.shape[0]])
        # print(f'temp_y.shape{temp_y.shape}')
        # temp_y

        temp_y_prediction = ml_model.predict(temp_x)
        # print(f'temp_y_prediction.shape{temp_y_prediction.shape}')
        # temp_y_prediction

    elif model_type == 'lstm':
        temp_x = temp_x.reshape([1, temp_x.shape[0], temp_x.shape[1]])
        gpu = torch.device(0)
        temp_x= Variable(torch.Tensor(temp_x))
        temp_x = temp_x.to(gpu)
        temp_y_prediction = ml_model(temp_x)
        temp_y_prediction = temp_y_prediction.cpu()
        temp_y_prediction = temp_y_prediction.detach().numpy()

    else: # if prophet
        temp_y = np.array(temp_y_all[stock])
        temp_y = temp_y.reshape([1, temp_y.shape[0]])
        temp_y_prediction = temp_y.copy()
        # print(temp_y_prediction)

    # print(type(temp_y_prediction))
    # print(temp_y_prediction.shape)
    temp_y_prediction = temp_y_prediction[temp_y_prediction.shape[0] - 1][temp_y_prediction.shape[1] - 1] / temp_y_prediction[0][0]
    return temp_y_prediction

def get_price(temp_price_dfr, temp_idx_date, code):
    temp_price = temp_price_dfr[(price_dfr['time'] == datetime.strptime(date_list[temp_idx_date], '%Y-%m-%d'))][code]
    return float(temp_price)

In [23]:
import json
recorder = {}
for HOLDING_NUM in range(5, 10, 10):
    holding_list = {}
    for idx_date in range(WINDOW, len(date_list) - FREQ):
        if idx_date % FREQ  == 0:  # 调仓日期
            print(f'Today is {date_list[idx_date]}. Trading...')
            if idx_date == WINDOW:  # 第一天
                holding_list[date_list[idx_date]] = {
                    'capital': INITIAL_MONEY,
                    'stock_code_list': [],
                    'stock_value_dict': {},
                    'return_rate': 0.0,
                    'prediction': {},
                }
            else:
                # 卖出持有股票
                holding_list[date_list[idx_date]] = {
                    'capital': holding_list[date_list[idx_date - 1]]['capital'],
                    'stock_code_list': [],
                    'stock_value_dict': {},
                    'return_rate': 0.0,
                    'prediction': {},
                }
            print('getting data for model...')
            x_all, y_all = cal_temp_x_y_all(fc_dfgr, price_dfr, idx_date)
            stock_score_dict = {}

            # 计算每支股票的得分
            print('predicting and scoring...')
            for stock_code in stock_list:
                # print(x_all.shape)
                # print(y_all.shape)
                y_prediction = cal_prediction(LSTM_MODEL, PCA_MODEL, x_all, y_all, stock_code, model_type='lstm')
                # y_prediction = cal_prediction(LSTM_MODEL, PCA_MODEL, x_all, y_all, stock_code, model_type='prophet')
                # y_prediction = cal_prediction(Lasso_MODEL, PCA_MODEL, x_all, y_all, stock_code, model_type='lasso')
                # print(f'y_prediction: {y_prediction}')
                stock_score_dict[stock_code] = np.sum(y_prediction)
                # y_prediction_list = y_prediction.tolist()
                # stock_score_dict[stock_code] = y_prediction[-1]

            # 获取得分最高的十支股票
            best_stock_list = stock_list[0:HOLDING_NUM]
            for stock_code in stock_list:
                for temp_best_stock in best_stock_list:
                    if (stock_score_dict[stock_code] > stock_score_dict[temp_best_stock]) and (stock_code not in best_stock_list):
                        best_stock_list.remove(temp_best_stock)
                        best_stock_list.append(stock_code)
            print(f'best_stock_list={best_stock_list}')

            # 更新持有信息
            print('Trading end, holding info:')
            holding_list[date_list[idx_date]]['stock_code_list'] = best_stock_list
            prediction_list = []
            for stock_code in best_stock_list:
                # print(f"np.float(stock_score_dict[stock_code]) = {np.float(stock_score_dict[stock_code])}")
                # print(f"type = {type(np.float(stock_score_dict[stock_code]))}")
                holding_list[date_list[idx_date]]['prediction'][stock_code] = np.float(stock_score_dict[stock_code])

            print(f"prediction: {holding_list[date_list[idx_date]]['prediction']}")
            for stock_code in best_stock_list:
                holding_list[date_list[idx_date]]['stock_value_dict'][stock_code] = holding_list[date_list[idx_date]]['capital'] / HOLDING_NUM
                # print(f"holding_list[date_list[idx_date]]['capital'] / HOLDING_NUM = {holding_list[date_list[idx_date]]['capital'] / HOLDING_NUM}")
                # print(f"type = {type(holding_list[date_list[idx_date]]['capital'] / HOLDING_NUM)}")

            with open(f'records/holding_list_HNUM{HOLDING_NUM}_idate{idx_date}', 'w', encoding='utf-8') as fp:
                json.dump(holding_list, fp, ensure_ascii=False, indent=4)
        else:  # 持有股票日期
            # 更新各股票的价值
            print(f'Today is {date_list[idx_date]}. Updating...')
            capital = 0
            holding_list[date_list[idx_date]] = holding_list[date_list[idx_date - 1]]
            for stock_code in holding_list[date_list[idx_date]]['stock_code_list']:
                if get_price(price_dfr, idx_date - 1, stock_code) == 0:
                    holding_list[date_list[idx_date]]['stock_value_dict'][stock_code] = holding_list[date_list[idx_date]]['stock_value_dict'][stock_code]
                    capital += holding_list[date_list[idx_date]]['stock_value_dict'][stock_code]
                else:
                    stock_value = get_price(price_dfr, idx_date, stock_code) / get_price(price_dfr, idx_date - 1, stock_code) * holding_list[date_list[idx_date]]['stock_value_dict'][stock_code]
                    holding_list[date_list[idx_date]]['stock_value_dict'][stock_code] = stock_value
                    capital += stock_value
            holding_list[date_list[idx_date]]['capital'] = capital
            holding_list[date_list[idx_date]]['return_rate'] = capital / INITIAL_MONEY
            print('Updating end, holding info:')
        print(holding_list[date_list[idx_date]])



Today is 2019-09-12. Trading...
getting data for model...
predicting and scoring...
best_stock_list=['002660.XSHE', '300093.XSHE', '600053.XSHG', '600409.XSHG', '600872.XSHG']
Trading end, holding info:
prediction: {'002660.XSHE': 101.72488403320312, '300093.XSHE': 66.80113220214844, '600053.XSHG': 58.06936264038086, '600409.XSHG': 13.950326919555664, '600872.XSHG': 20.171377182006836}
{'capital': 1000000.0, 'stock_code_list': ['002660.XSHE', '300093.XSHE', '600053.XSHG', '600409.XSHG', '600872.XSHG'], 'stock_value_dict': {'002660.XSHE': 200000.0, '300093.XSHE': 200000.0, '600053.XSHG': 200000.0, '600409.XSHG': 200000.0, '600872.XSHG': 200000.0}, 'return_rate': 0.0, 'prediction': {'002660.XSHE': 101.72488403320312, '300093.XSHE': 66.80113220214844, '600053.XSHG': 58.06936264038086, '600409.XSHG': 13.950326919555664, '600872.XSHG': 20.171377182006836}}
Today is 2019-09-16. Updating...
Updating end, holding info:
{'capital': 1013110.1860783345, 'stock_code_list': ['002660.XSHE', '300093.

D:\Applications\Anaconda3\install\envs\py3.7torch\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Updating end, holding info:
{'capital': 1006317.1707716839, 'stock_code_list': ['002660.XSHE', '300093.XSHE', '600053.XSHG', '600409.XSHG', '600872.XSHG'], 'stock_value_dict': {'002660.XSHE': 208966.1482159196, '300093.XSHE': 199178.8321167882, '600053.XSHG': 189300.41152263375, '600409.XSHG': 195163.10461192345, '600872.XSHG': 213708.674304419}, 'return_rate': 1.0063171707716838, 'prediction': {'002660.XSHE': 101.72488403320312, '300093.XSHE': 66.80113220214844, '600053.XSHG': 58.06936264038086, '600409.XSHG': 13.950326919555664, '600872.XSHG': 20.171377182006836}}
Today is 2019-10-28. Updating...
Updating end, holding info:
{'capital': 1025826.7725002262, 'stock_code_list': ['002660.XSHE', '300093.XSHE', '600053.XSHG', '600409.XSHG', '600872.XSHG'], 'stock_value_dict': {'002660.XSHE': 220677.03568161034, '300093.XSHE': 197810.2189781021, '600053.XSHG': 194014.21623643843, '600409.XSHG': 195950.50618672662, '600872.XSHG': 217374.79541734862}, 'return_rate': 1.0258267725002261, 'predic

D:\Applications\Anaconda3\install\envs\py3.7torch\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Updating end, holding info:
{'capital': 1025679.950578968, 'stock_code_list': ['600435.XSHG', '600876.XSHG', '600877.XSHG', '603398.XSHG', '603607.XSHG'], 'stock_value_dict': {'600435.XSHG': 198791.0514981751, '600876.XSHG': 217236.81674612383, '600877.XSHG': 184577.4659089785, '603398.XSHG': 189878.05882231702, '603607.XSHG': 235196.55760337348}, 'return_rate': 1.025679950578968, 'prediction': {'600435.XSHG': 29.542232513427734, '600876.XSHG': 5.610903739929199, '600877.XSHG': 12.323410987854004, '603398.XSHG': 15.67197036743164, '603607.XSHG': 20.621530532836914}}
Today is 2019-12-10. Updating...
Updating end, holding info:
{'capital': 1025277.8304461, 'stock_code_list': ['600435.XSHG', '600876.XSHG', '600877.XSHG', '603398.XSHG', '603607.XSHG'], 'stock_value_dict': {'600435.XSHG': 197558.951549019, '600876.XSHG': 217956.14395389243, '600877.XSHG': 182542.5163052454, '603398.XSHG': 193459.5374546147, '603607.XSHG': 233760.6811833285}, 'return_rate': 1.0252778304460999, 'prediction': 

...

D:\Applications\Anaconda3\install\envs\py3.7torch\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Updating end, holding info:
{'capital': 1214933.080168413, 'stock_code_list': ['002270.XSHE', '300309.XSHE', '300772.XSHE', '603456.XSHG', '688077.XSHG'], 'stock_value_dict': {'002270.XSHE': 255218.7455959623, '300309.XSHE': 226337.23666745177, '300772.XSHE': 208124.21808978668, '603456.XSHG': 220037.03990724383, '688077.XSHG': 305215.8399079684}, 'return_rate': 1.214933080168413, 'prediction': {'002270.XSHE': 13.461374282836914, '300309.XSHE': 13.189918518066406, '300772.XSHE': 10.333702087402344, '603456.XSHG': 6.544204235076904, '688077.XSHG': 5.231814384460449}}
Today is 2021-11-22. Updating...
Updating end, holding info:
{'capital': 1246827.6408781349, 'stock_code_list': ['002270.XSHE', '300309.XSHE', '300772.XSHE', '603456.XSHG', '688077.XSHG'], 'stock_value_dict': {'002270.XSHE': 254099.81127888156, '300309.XSHE': 228459.75124830595, '300772.XSHE': 205286.50001034213, '603456.XSHG': 218741.7050843967, '688077.XSHG': 340239.8732562087}, 'return_rate': 1.246827640878135, 'predicti

D:\Applications\Anaconda3\install\envs\py3.7torch\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Updating end, holding info:
{'capital': 1361803.9549083314, 'stock_code_list': ['002592.XSHE', '300727.XSHE', '300797.XSHE', '300943.XSHE', '605111.XSHG'], 'stock_value_dict': {'002592.XSHE': 221820.5163722322, '300727.XSHE': 244302.68578462087, '300797.XSHE': 288434.0216737432, '300943.XSHE': 381496.1804931046, '605111.XSHG': 225750.55058463066}, 'return_rate': 1.3618039549083314, 'prediction': {'002592.XSHE': 69.15760040283203, '300727.XSHE': 14.398489952087402, '300797.XSHE': 16.67364501953125, '300943.XSHE': 13.162912368774414, '605111.XSHG': 11.992843627929688}}
Today is 2022-01-05. Updating...
Updating end, holding info:
{'capital': 1377139.6304552485, 'stock_code_list': ['002592.XSHE', '300727.XSHE', '300797.XSHE', '300943.XSHE', '605111.XSHG'], 'stock_value_dict': {'002592.XSHE': 222762.4293929211, '300727.XSHE': 240236.62166615287, '300797.XSHE': 283635.0620475019, '300943.XSHE': 415525.16693372466, '605111.XSHG': 214980.35041494813}, 'return_rate': 1.3771396304552486, 'predic

D:\Applications\Anaconda3\install\envs\py3.7torch\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
